In [ ]:
class optimazition_problem:
    def __init__(self,initial_state):
        self.initial_state= initial_state
        pass
    @property
    def goal_state(self):
        ##in this code we will retrieve data from the dataset according to some parameters of the inital_state
        ##suggested solution, for the data preprocessing,we will do two groupings:
        ## in the first one: we will group data according to the parameters that are related the water need and the soil_moisture,then we will calculate average values for soil_moisture,irrigation_frequency and wue under those conditions
        ## in the second one: we will group data accroding to the parameters that are related directly to the fertilizer need,then we will calculate average values for fertilizer_usage
        ## user inputs area and crop density==> crop_yield==> w7
       pass

    def priorities(self):
        """Calculate the base priorities before normalization."""
        water_priority = 0.33
        fertilizer_priority = 0.33
        irrigation_frequency_priority = 0.33
        
        # --- Adjust for crop-specific factors ---
        
        # Growth stage adjustments
        if self.growth_stage==1:  # Germination/early growth
            water_priority += 0.1
            fertilizer_priority += 0.05
            irrigation_frequency_priority += 0.1
        elif  self.growth_stage==2 :  # Vegetative growth
            fertilizer_priority += 0.15
            water_priority += 0.05
        elif  self.growth_stage==3 :  # Flowering/fruiting
            water_priority += 0.15
            fertilizer_priority += 0.1
            irrigation_frequency_priority += 0.05
        else:  # Maturation/ripening
            water_priority -= 0.05
            fertilizer_priority -= 0.1
        
        
        
        # --- Adjust for environmental factors ---
        
        # Temperature and humidity adjustments
        heat_stress = max(0, min(1, (self.temperature - 25) / 15))  # Normalized heat stress (25-40°C range)
        water_priority += heat_stress * 0.2
        irrigation_frequency_priority += heat_stress * 0.15
        
        water_priority -= (self.humidity / 100) * 0.1  # Higher humidity reduces water priority
        
        # Rainfall forecast adjustments
        rainfall_factor = min(1, self.rainfall_forecast / 25)  # Normalize expected rainfall
        water_priority -= rainfall_factor * 0.2
        irrigation_frequency_priority -= rainfall_factor * 0.3
        
        # Soil moisture adjustments
        drought_factor = max(0, 1 - (self.soil_moisture / 0.7))  # Below 70% of field capacity
        water_priority += drought_factor * 0.25
        irrigation_frequency_priority += drought_factor * 0.2
        
        # Soil type adjustments
        if self.soil_type == "1":
            irrigation_frequency_priority =+0.15
            water_priority += 0.1
        elif self.soil_type == "3":
            irrigation_frequency_priority -= 0.1
            water_priority -= 0.05
            fertilizer_priority += 0.05  # Clay holds nutrients better
        
        # --- Adjust for resource constraints ---
        
        # Water availability adjustments
        water_priority -= self.water_availability * 0.1  # Abundant water reduces priority
        if self.water_availability < 0.3:  # Water scarcity
            water_priority += 0.2  # Critical factor in water-scarce conditions
            irrigation_frequency_priority += 0.1  # More precise irrigation timing needed
            water_priority += 0.1  # Make each irrigation count
        
        # --- Adjust for practical implementation factors ---
        
        # Irrigation system efficiency
        if self.irrigation_system == "drip":
            irrigation_frequency_priority += 0.1
            water_priority -= 0.15  # Drip systems use less water
        elif self.irrigation_system == "flood":
            irrigation_frequency_priority -= 0.2
            water_priority += 0.15  # Flood irrigation uses more water
        
        # Ensure priorities are non-negative
        water_priority = max(0.1, water_priority)
        fertilizer_priority = max(0.1, fertilizer_priority)
        irrigation_frequency_priority = max(0.1, irrigation_frequency_priority)
        
        return water_priority, fertilizer_priority, irrigation_frequency_priority
    

    # 1. Distance from optimal range (penalty if out of range)
    def penalty(self,var, value):
        min_val, max_val = self.optimal_ranges[var]
        if value < min_val:
            return (min_val - value)**2 ##(squaring differences will make the heuristic values mor differentiable and smooth)
        elif value > max_val:
            return (value - max_val)**2
        else:
            return 0  # in optimal range

    def heuristic(self,state):
    
       # 2. Compute total deviation
        deviation_score = (
        self.penalty('soil_moisture', state['soil_moisture']) +
        self.penalty('N', state['N']) +
        self.penalty('P', state['P']) +
        self.penalty('K', state['K'])
        )

      # 3. Add cost weighting
        cost_penalty = (
        self.priorities['water_priority'] * state['water_used'] +
        self.priorities['fertilizer_priority'] * state['fertilizer_used']
       )

      # 4. Optionally reward good WUE
        wue_bonus = 0
        if 'WUE' in state and 'WUE' in self.optimal_ranges['WUE']:
            wue_penalty = self.penalty('WUE', self.state['WUE'])
            deviation_score += wue_penalty

        return deviation_score + cost_penalty
    
    def get_actions(self):
        pass

    def apply_action(self,node,action):
        water_added, fertilizer_added = action 
        soil_type = str(node.state['soil_type'])  # ensure it's a string to match keys
    
        new_node = node.copy()  # deep copy to avoid mutating original

       # === Apply water ===
        if water_added > 0:
        # Get soil moisture increase per liter for this soil type
            moisture_per_L = self.transition_model["add_water"]["soil_moisture_increase_per_L"][soil_type]
            delta_moisture = water_added * moisture_per_L
            new_node['soil_moisture'] += delta_moisture
        
        # Update NPK uptake from added moisture
        uptake_per_1pct = self.transition_model["add_water"]["npk_uptake_increase_per_1_percent_moisture"][soil_type]
        for nutrient in ['N', 'P', 'K']:
            new_node[nutrient] += delta_moisture * uptake_per_1pct[nutrient]
            new_node[nutrient] = min(new_node[nutrient], 1.0)  # cap at 100%

        # Track water usage
        new_node['water_used'] += water_added

       # === Apply fertilizer ===
        if fertilizer_added > 0:
            npk_gain = self.transition_model["apply_fertilizer"]["npk_availability_increase"][soil_type]
            for nutrient in ['N', 'P', 'K']:
                new_node[nutrient] += fertilizer_added * npk_gain[nutrient]
                new_node[nutrient] = min(new_node[nutrient], 1.0)  # cap at 100%

        new_node['fertilizer_used'] += fertilizer_added

        return new_node

    def expand_node(self,node):
        children = []
        actions = self.get_actions()
        for action in actions:
            child = self.apply_action(node,action)
            child.g = node.g + action['water_amount']*self.water_cost + action['fertilizer_amount']*self.fertilizer_cost
            h = self.heuristic(child.state)
            child.f = child.g + h
            children.append(child)
        return children

    transtion_model = {
    "add_water": {
        "units": "1 L/m²",
        "soil_moisture_increase_per_L": {
            "1": 1.0,  # % increase in soil moisture
            "2": 0.6,
            "3": 0.3
        },
        "npk_uptake_increase_per_1_percent_moisture": {
            "1": {
                "N": 0.02,  # +2% N uptake per 1% soil moisture increase
                "P": 0.015,
                "K": 0.018
            },
            "2": {
                "N": 0.025,
                "P": 0.02,
                "K": 0.022
            },
            "3": {
                "N": 0.015,
                "P": 0.025,
                "K": 0.02
            }
        }
    },

    "apply_fertilizer": {
        "units": "1 kg/m²",
        "npk_availability_increase": {
            "1": {
                "N": 0.15,
                "P": 0.10,
                "K": 0.12
            },
            "2": {
                "N": 0.20,
                "P": 0.18,
                "K": 0.20
            },
            "3": {
                "N": 0.12,
                "P": 0.25,
                "K": 0.18
            }
        }
    }
   }



state is a dictionnary defined as follows:
{soil_moisture:x,soil_type:"string",water_allocated:x,fertilizer_allocated:x,irrigation_frequency:x,...,all the columns of the dataset}the 

user needs to input the water_cost and the fertilizer_cost, the water_availability,and the fertilizer availability

In [ ]:
class node:
    def __init__(self,state,parent=None,g=0,f=0,):
        self.state = state
        self.parent = parent 
        self.f=f
        self.g=g
    